## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

Using TensorFlow backend.


In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
print(len(x_train[0]))

218


In [5]:
# print(x_train) # is a list of list 

In [6]:
# print(x_train[0])

In [7]:
print(len(x_train[1]))

189


In [8]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [9]:
print(x_train.shape)

(25000, 30)


In [10]:
x_train[0,:]  #Here's what an example sequence looks like

array([  18,   51,   36,   28,  224,   92,   25,  104,    4,  226,   65,
         16,   38, 1334,   88,   12,   16,  283,    5,   16, 4472,  113,
        103,   32,   15,   16, 5345,   19,  178,   32], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [11]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [12]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [13]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 14s 541us/step - loss: 0.6673 - accuracy: 0.6154 - val_loss: 0.6192 - val_accuracy: 0.6811
Epoch 2/10
25000/25000 [==============================] - 13s 533us/step - loss: 0.5756 - accuracy: 0.7086 - val_loss: 0.5582 - val_accuracy: 0.7132
Epoch 3/10
25000/25000 [==============================] - 11s 438us/step - loss: 0.5150 - accuracy: 0.7514 - val_loss: 0.5249 - val_accuracy: 0.7341
Epoch 4/10
25000/25000 [==============================] - 12s 470us/step - loss: 0.4747 - accuracy: 0.7790 - val_loss: 0.5027 - val_accuracy: 0.7493
Epoch 5/10
25000/25000 [==============================] - 14s 569us/step - loss: 0.4444 - accuracy: 0.7967 - val_loss: 0.4857 - val_accuracy: 0.7613
Epoch 6/10
25000/25000 [==============================] - 15s 589us/step - loss: 0.4207 - accuracy: 0.8087 - val_loss: 0.4716 - val_accuracy: 0.7703
Epoch 7/10
25000/25000 [==============================]

In [15]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 8s 307us/step
Test score: 0.4489815320587158
Test accuracy: 0.7866799831390381


## Exercise
### Your Turn

Now do it yourself:
- Prepare the data to use sequences of length 80 rather than length 30.  Did it improve the performance?
- Try different values of the "max_features".  Can you improve the performance?
- Try smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [16]:
# Please provide your code here

max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this
batch_size = 32

In [17]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [18]:

rnn_hidden_dim = 10
word_embedding_dim = 50
model_rnn2 = Sequential()
model_rnn2.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn2.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn2.add(Dense(1, activation='sigmoid'))

In [19]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn2.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [20]:
model_rnn2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 10)                610       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,000,621
Trainable params: 1,000,621
Non-trainable params: 0
_________________________________________________________________


In [21]:
model_rnn2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 88s 4ms/step - loss: 0.6503 - accuracy: 0.6132 - val_loss: 0.5895 - val_accuracy: 0.6872
Epoch 2/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.5360 - accuracy: 0.7298 - val_loss: 0.5296 - val_accuracy: 0.7300
Epoch 3/10
25000/25000 [==============================] - 86s 3ms/step - loss: 0.4702 - accuracy: 0.7749 - val_loss: 0.4860 - val_accuracy: 0.7611
Epoch 4/10
25000/25000 [==============================] - 87s 3ms/step - loss: 0.4298 - accuracy: 0.8016 - val_loss: 0.4659 - val_accuracy: 0.7745
Epoch 5/10
25000/25000 [==============================] - 96s 4ms/step - loss: 0.4054 - accuracy: 0.8144 - val_loss: 0.4530 - val_accuracy: 0.7827
Epoch 6/10
25000/25000 [==============================] - 98s 4ms/step - loss: 0.3881 - accuracy: 0.8242 - val_loss: 0.4489 - val_accuracy: 0.7856
Epoch 7/10
25000/25000 [==============================] - 96s 4ms/st

In [22]:
score, acc = model_rnn2.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 9s 358us/step
Test score: 0.45190062059402464
Test accuracy: 0.7883999943733215
